VN2 Advanced Round 1 — Model-based Ordering Notebook

Purpose
- Produce a model-based submission using per-SKU routing (SES/Croston/TBATS) and base-stock policy.
- Keep the notebook focused: minimal demo, maximum actionability.

Flow
1) Setup and file handles
2) Project imports (policy helpers)
3) Load and reshape sales/availability
4) Routing features (mean, zero_frac, season strength)
5) Forecasters (SES, Croston-SBA, TBATS)
6) Per-SKU routing → μ̂, σ̂ table (`demand_stats_modelled`)
7) Build submission `orders_round1_advanced_modelled.csv`



Global warning suppression (optional)

- To keep outputs readable. Remove if you need warnings for debugging.



In [1]:
import os, warnings
os.environ["PYTHONWARNINGS"] = "ignore"
warnings.filterwarnings("ignore")
warnings.showwarning = lambda *args, **kwargs: None


Setup and file handles

- Imports core libraries, sets seaborn theme, defines `files` mapping for the five inputs.



In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sns
sns.set_theme(style="whitegrid")

DATA_DIR = Path("../data").resolve()
files = {
    "initial": DATA_DIR / "Week 0 - 2024-04-08 - Initial State.csv",
    "sales": DATA_DIR / "Week 0 - 2024-04-08 - Sales.csv",
    "in_stock": DATA_DIR / "Week 0 - In Stock.csv",
    "master": DATA_DIR / "Week 0 - Master.csv",
    "template": DATA_DIR / "Week 0 - Submission Template.csv",
}
files


{'initial': PosixPath('/Users/senoni/noni/vn2inventory/data/Week 0 - 2024-04-08 - Initial State.csv'),
 'sales': PosixPath('/Users/senoni/noni/vn2inventory/data/Week 0 - 2024-04-08 - Sales.csv'),
 'in_stock': PosixPath('/Users/senoni/noni/vn2inventory/data/Week 0 - In Stock.csv'),
 'master': PosixPath('/Users/senoni/noni/vn2inventory/data/Week 0 - Master.csv'),
 'template': PosixPath('/Users/senoni/noni/vn2inventory/data/Week 0 - Submission Template.csv')}

Project imports

- Adds project root to `sys.path` and imports policy helpers used for base-stock computation.



In [3]:
import sys
PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
from vn2inventory.policy import compute_orders, _inv_normal_cdf


Load and reshape inputs

- Reads wide sales/availability, melts to long format, coerces types.



In [4]:
sales_wide = pd.read_csv(files["sales"])  
avail_wide = pd.read_csv(files["in_stock"]) 

id_cols = ["Store","Product"]
sales_long = sales_wide.melt(id_vars=id_cols, var_name="Week", value_name="SalesQty")
sales_long["SalesQty"] = pd.to_numeric(sales_long["SalesQty"], errors="coerce").fillna(0.0)

avail_long = avail_wide.melt(id_vars=id_cols, var_name="Week", value_name="InStock")
avail_long["InStock"] = avail_long["InStock"].astype(str).str.lower().map({"true": True, "false": False})

sales_long.head(), avail_long.head()


(   Store  Product        Week  SalesQty
 0      0      126  2021-04-12       0.0
 1      0      182  2021-04-12       0.0
 2      1      124  2021-04-12      13.0
 3      2      124  2021-04-12       5.0
 4      2      126  2021-04-12       0.0,
    Store  Product        Week  InStock
 0      0      126  2021-04-12     True
 1      0      182  2021-04-12    False
 2      1      124  2021-04-12     True
 3      2      124  2021-04-12     True
 4      2      126  2021-04-12    False)

Routing features and thresholds

- Compute per-SKU features for model routing: `mean`, `zero_frac`, `season_strength` (STL, period=52), `length`.
- Thresholds (tune as needed): ZERO_FRAC_THR=0.5, LOW_MEAN_THR=1.5, SEASON_STRENGTH_THR=0.3, MIN_LEN_TBATS=60.



In [5]:
from statsmodels.tsa.seasonal import STL

ZERO_FRAC_THR = 0.5
LOW_MEAN_THR = 1.5
SEASON_STRENGTH_THR = 0.3
MIN_LEN_TBATS = 60

series_list = []
for (s,p), g in sales_long.groupby(["Store","Product"]):
    y = g.set_index("Week")["SalesQty"].astype(float)
    y.index = pd.to_datetime(y.index)
    y = y.asfreq("W-MON", fill_value=0.0)
    zero_frac = (y == 0).mean()
    mean_y = y.mean()
    seas_strength = 0.0
    try:
        if len(y) >= MIN_LEN_TBATS:
            seas_strength = STL(y, period=52, robust=True).fit().seasonal.var() / (y.var() + 1e-9)
    except Exception:
        seas_strength = 0.0
    series_list.append({
        "Store": s, "Product": p,
        "mean": float(mean_y), "zero_frac": float(zero_frac), "season_strength": float(seas_strength),
        "length": int(len(y))
    })

features_df = pd.DataFrame(series_list).set_index(["Store","Product"])  
features_df.head()


mean  zero_frac  season_strength  length
Store Product                                              
0     126      2.076433   0.464968         0.745242     157
      182      0.643312   0.585987         0.679455     157
1     124      5.299363   0.235669         0.672974     157
2     124      9.273885   0.000000         0.944765     157
      126      2.433121   0.420382         0.828707     157

Forecasters for routing

- SES: general-purpose smoother
- Croston-SBA: intermittent demand
- TBATS: seasonal/high-volume (used selectively)



In [6]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from tbats import TBATS


def fc_naive_last(train: pd.Series, horizon: int) -> np.ndarray:
    return np.repeat(train.iloc[-1] if len(train) else 0.0, horizon)


def fc_ses(train: pd.Series, horizon: int) -> np.ndarray:
    if len(train) < 2:
        return fc_naive_last(train, horizon)
    model = SimpleExpSmoothing(train.astype(float)).fit(optimized=True)
    return model.forecast(horizon).values


def fc_croston_sba(train: pd.Series, horizon: int) -> np.ndarray:
    y = train.values.astype(float)
    alpha = 0.1
    z, p = 0.0, 0.0
    q = 0
    for v in y:
        if v > 0:
            if z == 0:
                z = v
                p = 1
            else:
                z = alpha * v + (1 - alpha) * z
                p = alpha * (q + 1) + (1 - alpha) * p
            q = 0
        else:
            q += 1
    rate = z / p if p > 0 else 0.0
    return np.repeat(rate * (1 - alpha / 2), horizon)


def fc_tbats(train: pd.Series, horizon: int) -> np.ndarray:
    y = train.astype(float).values
    if len(y) < 10:
        return fc_naive_last(train, horizon)
    estimator = TBATS(use_arma_errors=False, use_box_cox=False)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", FutureWarning)
        model = estimator.fit(y)
        fc = model.forecast(steps=horizon)
    return np.asarray(fc, dtype=float)



Per-SKU routing to forecasts and uncertainty

- For each `(Store, Product)`, select a model using features in `features_df`.
- Estimate weekly mean demand (μ̂) and uncertainty (σ̂) for use in the base-stock policy.
- Output: `demand_stats_modelled` with `mean_demand`, `std_demand`, and `model`.


In [7]:
rows = []
for (s,p), g in sales_long.groupby(["Store","Product"]):
    y = g.set_index("Week")["SalesQty"].astype(float)
    y.index = pd.to_datetime(y.index)
    y = y.asfreq("W-MON", fill_value=0.0)
    feat = features_df.loc[(s,p)]

    # Routing
    if feat["zero_frac"] >= ZERO_FRAC_THR and feat["mean"] < LOW_MEAN_THR:
        model_name = "croston"
        mu = float(y.mean())  # keep central demand level for intermittent
        res = y - mu
    elif feat["season_strength"] >= SEASON_STRENGTH_THR and feat["mean"] >= 3 and feat["length"] >= MIN_LEN_TBATS:
        model_name = "tbats"
        try:
            est = TBATS(use_arma_errors=False, use_box_cox=False)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", FutureWarning)
                mdl = est.fit(y.values)
                fc = mdl.forecast(steps=4)
            mu = float(np.mean(fc))
            res = y - pd.Series(mdl.y_hat, index=y.index)
        except Exception:
            model_name = "ses_fallback"
            mdl = SimpleExpSmoothing(y).fit(optimized=True)
            fc = mdl.forecast(4)
            mu = float(np.mean(fc))
            res = y - mdl.fittedvalues
    else:
        model_name = "ses"
        mdl = SimpleExpSmoothing(y).fit(optimized=True)
        fc = mdl.forecast(4)
        mu = float(np.mean(fc))
        res = y - mdl.fittedvalues

    sigma = float(np.std(res.dropna(), ddof=1)) if res.notna().any() else float(np.std(y, ddof=1))
    rows.append({"Store": s, "Product": p, "model": model_name, "mean_demand": mu, "std_demand": sigma})

demand_stats_modelled = pd.DataFrame(rows).set_index(["Store","Product"])  
demand_stats_modelled.head()


model  mean_demand  std_demand
Store Product                                  
0     126          ses     1.144080    2.424456
      182      croston     0.643312    0.933787
1     124        tbats     7.849874    4.346528
2     124        tbats     8.537111    4.553976
      126          ses     1.910470    3.114283

Build submission

- Align to the platform index (Submission Template)
- Build current state from Initial State (`on_hand = End Inventory`, `on_order = W+1 + W+2`)
- Call base-stock policy with `demand_stats_modelled` and write `submissions/orders_round1_advanced_modelled.csv`



In [8]:
from pathlib import Path as _P

index_df = pd.read_csv(files['template'])[['Store','Product']].set_index(['Store','Product'])
init = pd.read_csv(files['initial'])
state = init[["Store","Product","End Inventory","In Transit W+1","In Transit W+2"]].copy()
state.rename(columns={"End Inventory":"on_hand"}, inplace=True)
state['on_order'] = state[["In Transit W+1","In Transit W+2"]].sum(axis=1)
state = state[["Store","Product","on_hand","on_order"]].set_index(["Store","Product"])  

orders = compute_orders(
    index_df=index_df,
    demand_stats=demand_stats_modelled,
    current_state=state,
    lead_time_weeks=2,
    review_period_weeks=1,
    shortage_cost_per_unit=1.0,
    holding_cost_per_unit_per_week=0.2,
)

SUB = _P("../submissions").resolve(); SUB.mkdir(exist_ok=True)
submission = index_df.copy(); submission['order_qty'] = orders.values
out_path = SUB / "orders_round1_advanced_modelled.csv"
submission.to_csv(out_path)
out_path


PosixPath('/Users/senoni/noni/vn2inventory/submissions/orders_round1_advanced_modelled.csv')